In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
data = np.load('/content/drive/MyDrive/Xy_train.npz')
X_train = pd.DataFrame(data['X'])
y_train = pd.DataFrame(data['y'])
data = np.load("/content/drive/MyDrive/X_test.npz")
X_test = pd.DataFrame(data['X'])

In [3]:
X = X_train.append(X_test)
X.index = range(X.shape[0])

<ipython-input-3-e718d2be2b37>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X_train.append(X_test)


In [4]:
X.columns = [str(i) for i in range(X.shape[1])]
y_train.columns = ['y']

In [ ]:
for i in range(X.shape[1]):
  if (len(X[str(i)].unique()) < 100):
    X[str(i)] = X[str(i)].astype('object')

In [ ]:
X

In [ ]:
sum(X.isna().sum())

In [ ]:
n_total = X_train.shape[0]
train_indices, valid_indices = train_test_split(range(n_total), test_size=0.2, random_state=42)
test_indices = [int(i) for i in range(400000,500000)]

In [ ]:
from sklearn.preprocessing import LabelEncoder
categorical_columns = []
categorical_dims =  {}
for col in X.columns[X.dtypes == 'object']:
    print(col, X[col].nunique())
    l_enc = LabelEncoder()
    X[col] = l_enc.fit_transform(X[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

In [ ]:
cat_idxs = [int(i) for i in categorical_columns]
cat_dims = [categorical_dims[f] for f in categorical_columns]

In [ ]:
features = list(X_train.columns)

In [ ]:
trainX = X[features].values[train_indices]
trainy = y_train['y'].values[train_indices]
testX = X[features].values[test_indices]

valX = X[features].values[valid_indices]
valy = y_train['y'].values[valid_indices]

In [ ]:
!pip install pytorch-tabnet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
clf = TabNetClassifier(
    n_d=64, n_a=64, n_steps=3,
    gamma=1.5, seed = 42,
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-2),
    scheduler_params = {"gamma": 0.9,
                     "step_size": 10},
    scheduler_fn=torch.optim.lr_scheduler.StepLR
)
clf.fit(
  trainX, trainy,
  eval_set=[(trainX, trainy),(valX, valy)],
  eval_metric = ['balanced_accuracy'],
  max_epochs = 200,
  compute_importance = False,
  batch_size= 1024,
  virtual_batch_size=1024,
  patience = 30,
  )
saved_filename = clf.save_model('test_model')

In [ ]:
y_pred = clf.predict(testX)

In [ ]:
import pandas as pd
from collections import Counter
print(Counter(y_pred).items())
a = pd.read_csv('(1)y679zhao.csv')
print(Counter(a['Predicted']).items())

In [ ]:
import pandas as pd
#y_pred = [int(i) for i in y_pred]
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['Predicted']
y_pred.index = y_pred.index
y_pred.index.name = 'ID'
y_pred.to_csv('y679zhao.csv')